# Модель стоимости жилья в Магнитогорске

**id для демонстрации на собеседовании**
id = 3, 37, 82

## Введение

**Задача**

Построить математическую модель стоимости жилья в зависимости от параметров этого жилья.


Модель должна иметь REST API. На вход модели подаются параметры квартиры в формате JSON на выходе получается цена квартиры в формате JSON.

Испытание проводится в режиме демонстрации экрана на собеседовании. Модель тестируется на 3х квартирах на выбор кандидата. (Просьба подготовить исходные данные (запросы) заранее)


**Данные**


В качестве источника исходных данных предлагается использовать данные сайта магнитогорской недвижимости www.citystar.ru.

Размер выборки исходных данных не имеет значения, однако она должна быть представительной (не следует делать ее слишком большой, т.к. задача тестовая).

Данные должны быть загружены в базу данных.

## Импорт библиотек и загрузка данных

In [87]:
import sqlite3
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error

RANDOM_STATE = 12345

In [2]:
# подключаемся к базе данных
cnx = sqlite3.connect('db/magnitogorsk.db')

In [3]:
# загружаем данные
data = pd.read_sql_query("SELECT * FROM offers", cnx)

In [4]:
# закрываем соединение с базой данных
cnx.close()

## Первичное знакомство с данными

In [5]:
def first_look(df, num_of_srtings=5):
    print('Общая информация')
    display(df.info())
    
    print(f'Первые {num_of_srtings} строк(и) данных')
    display(df.head(num_of_srtings))
    
    print('Основные статистические характеристики данных')
    display(df.describe())
    print('Количество пропусков:')
    print(df.isna().sum())
    print()
    
    print('Количество дубликатов:', df.duplicated().sum())

In [6]:
first_look(data)

Общая информация
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456 entries, 0 to 455
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              456 non-null    int64  
 1   type            445 non-null    object 
 2   district        254 non-null    object 
 3   adress          456 non-null    object 
 4   floor           456 non-null    object 
 5   total_square    456 non-null    float64
 6   living_square   456 non-null    float64
 7   kitchen_square  456 non-null    float64
 8   price           456 non-null    int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 32.2+ KB


None

Первые 5 строк(и) данных


,id,type,district,adress,floor,total_square,living_square,kitchen_square,price
0,1,Трехкомнатная улучшенная,Орджоникидзевский,Ленина пр-т 145/2,1/5,64.0,43.0,8.0,3750
1,2,Трехкомнатная,Ленинский,Октябрьская 12,2/5,87.2,60.0,9.0,8300
2,3,Однокомнатная нестандартная,Орджоникидзевский,Ленина пр-т 135,6/14,36.1,20.0,9.0,3330
3,4,Трехкомнатная нестандартная,Орджоникидзевский,Ленина пр-т 129,5/16,105.0,75.0,14.0,7700
4,5,Двухкомнатная улучшенная,Орджоникидзевский,Сиреневый проезд 12,7/9,50.6,43.0,9.0,3800


Основные статистические характеристики данных


,id,total_square,living_square,kitchen_square,price
count,456.00000,456.000000,456.000000,456.000000,456.000000
mean,228.50000,53.514912,32.120175,8.771250,3744.561404
std,131.78012,21.751910,17.343334,4.007841,1631.107124
min,1.00000,14.100000,0.000000,0.000000,0.000000
25%,114.75000,40.175000,19.000000,6.000000,2700.000000
50%,228.50000,50.000000,30.000000,8.050000,3500.000000
75%,342.25000,65.000000,43.000000,10.000000,4600.000000
max,456.00000,220.000000,150.000000,30.000000,10000.000000


Количество пропусков:
id                  0
type               11
district          202
adress              0
floor               0
total_square        0
living_square       0
kitchen_square      0
price               0
dtype: int64

Количество дубликатов: 0


### Цена

In [7]:
data[data['price'] == 0]

,id,type,district,adress,floor,total_square,living_square,kitchen_square,price
73,74,Однокомнатная,None,Первомайская 16,2/4,42.6,20.0,15.0,0
141,142,Однокомнатная,None,Труда 47,4/9,32.0,19.0,6.0,0
399,400,Трехкомнатная,Орджоникидзевский,ул Репина 12,1/1,55.0,39.0,6.0,0


Для трех квартир цена не указана, поэтому удалим эти строки, так как они не подходят ни для обучения модели ни для контроля качества.

In [8]:
# контроль размерности
data.shape

(456, 9)

In [9]:
data = data[data['price'] > 0]

In [10]:
# контроль размерности
data.shape

(453, 9)

### Этаж

In [11]:
data['floor'].isna().sum()

0

In [12]:
def get_floor_num(row):
    return int(row[:row.find('/')])

In [13]:
data['floor_num'] = data['floor'].apply(get_floor_num)

In [14]:
data['floor_num'].isna().sum()

0

In [15]:
def get_total_floors(row):
    return int(row[row.find('/')+1:])

In [16]:
data['total_floors'] = data['floor'].apply(get_total_floors)

In [17]:
data['total_floors'].isna().sum()

0

In [18]:
data.head()

,id,type,district,adress,floor,total_square,living_square,kitchen_square,price,floor_num,total_floors
0,1,Трехкомнатная улучшенная,Орджоникидзевский,Ленина пр-т 145/2,1/5,64.0,43.0,8.0,3750,1,5
1,2,Трехкомнатная,Ленинский,Октябрьская 12,2/5,87.2,60.0,9.0,8300,2,5
2,3,Однокомнатная нестандартная,Орджоникидзевский,Ленина пр-т 135,6/14,36.1,20.0,9.0,3330,6,14
3,4,Трехкомнатная нестандартная,Орджоникидзевский,Ленина пр-т 129,5/16,105.0,75.0,14.0,7700,5,16
4,5,Двухкомнатная улучшенная,Орджоникидзевский,Сиреневый проезд 12,7/9,50.6,43.0,9.0,3800,7,9


Добавим столбец с информацией о том, является ли этаж квартиры первым.

In [19]:
data['is_first_floor'] = data['floor_num'] == 1
data['is_first_floor'] = data['is_first_floor'].astype('int')

Добавим столбец с информацией о том, является ли этаж квартиры последним.

In [20]:
data['is_last_floor'] = data['floor_num'] == data['total_floors']
data['is_last_floor'] = data['is_last_floor'].astype('int')

In [21]:
data.head(10)

,id,type,district,adress,floor,total_square,living_square,kitchen_square,price,floor_num,total_floors,is_first_floor,is_last_floor
0,1,Трехкомнатная улучшенная,Орджоникидзевский,Ленина пр-т 145/2,1/5,64.0,43.0,8.0,3750,1,5,1,0
1,2,Трехкомнатная,Ленинский,Октябрьская 12,2/5,87.2,60.0,9.0,8300,2,5,0,0
2,3,Однокомнатная нестандартная,Орджоникидзевский,Ленина пр-т 135,6/14,36.1,20.0,9.0,3330,6,14,0,0
3,4,Трехкомнатная нестандартная,Орджоникидзевский,Ленина пр-т 129,5/16,105.0,75.0,14.0,7700,5,16,0,0
4,5,Двухкомнатная улучшенная,Орджоникидзевский,Сиреневый проезд 12,7/9,50.6,43.0,9.0,3800,7,9,0,0
5,6,Двухкомнатная улучшенная,Орджоникидзевский,Ленина пр-т 141,4/9,49.7,35.0,9.0,4000,4,9,0,0
6,7,Двухкомнатная,Правобережный,Советской Армии 9,1/5,43.8,28.6,6.0,3200,1,5,1,0
7,8,Однокомнатная брежневка,Правобережный,Карла Маркса 99,4/9,31.0,17.0,6.0,2670,4,9,0,0
8,9,Однокомнатная хрущевка,Ленинский,Ленинградская 37а,2/5,31.0,19.0,6.0,2650,2,5,0,0
9,10,Однокомнатная хабаровский вариант,Орджоникидзевский,Сиреневый проезд 14/2,6/6,37.0,19.0,8.0,2990,6,6,0,1


### Тип квартиры

В столбце с типом квартиры хранятся данные о количестве комнат и типе квартиры. Разделим их и сохраним в двух разных столбцах.

In [22]:
data['type'].isna().sum()

11

In [23]:
data['type'].isna().sum() / data.shape[0] * 100

2.4282560706401766

In [24]:
data[data['type'].isna()]

,id,type,district,adress,floor,total_square,living_square,kitchen_square,price,floor_num,total_floors,is_first_floor,is_last_floor
59,60,None,None,Ленина пр-т 212а,1/1,18.4,12.0,5.0,1100,1,1,1,1
83,84,None,None,Карла Маркса 233,8/10,70.0,50.0,7.0,4860,8,10,0,0
97,98,None,Орджоникидзевский,Завенягина 1,5/9,65.1,44.0,8.0,4750,5,9,0,0
140,141,None,None,Торфяная 5/2,1/2,46.8,0.0,0.0,4950,1,2,1,0
161,162,None,None,Карла Маркса 119/1,2/5,41.0,0.0,6.0,3600,2,5,0,0
162,163,None,None,Карла Маркса 117,2/5,41.0,26.0,6.0,3400,2,5,0,0
163,164,None,None,ул Жукова 17/1,4/9,40.1,18.0,9.0,3100,4,9,0,0
176,177,None,None,Западное шоссе 101,2/3,68.0,40.0,0.0,6000,2,3,0,0
355,356,None,None,ул Лесопарковая 93/1,10/10,41.2,28.0,12.5,3150,10,10,0,1
365,366,None,None,Ленина пр-т 114/4,5/14,40.0,18.0,13.0,2620,5,14,0,0


In [25]:
data = data[data['type'].notna()]

In [26]:
data['type'].unique()

array(['Трехкомнатная улучшенная', 'Трехкомнатная ',
       'Однокомнатная нестандартная', 'Трехкомнатная нестандартная',
       'Двухкомнатная улучшенная', 'Двухкомнатная ',
       'Однокомнатная брежневка', 'Однокомнатная хрущевка',
       'Однокомнатная хабаровский вариант', 'Однокомнатная ',
       'Трехкомнатная старой планировки', 'Двухкомнатная нестандартная',
       'Однокомнатная свердловский вариант', 'Двухкомнатная брежневка',
       'Трехкомнатная свердловский вариант',
       'Четырехкомнатная распашонка', 'Двухкомнатная гребенка',
       'Двухкомнатная раздельная', 'Двухкомнатная хрущевка',
       'Двухкомнатная старой планировки',
       'Однокомнатная сталинский вариант', 'Трехкомнатная брежневка',
       'Двухкомнатная сталинский вариант', 'Многокомнатная ',
       'Четырехкомнатная ', 'Однокомнатная улучшенная',
       'Двухкомнатная хабаровский вариант', 'Однокомнатная малосемейка',
       'Четырехкомнатная сталинский вариант', 'Трехкомнатная раздельная',
       'Тре

In [27]:
def get_num_of_rooms(row):
    return row[: row.find(' ')] if row.find(' ') > 0 else None

In [28]:
def get_flat_type(row):
    return 'неизвестно' if (len(row) - row.find(' ') == 1) or (row.find(' ') == -1) else row[row.find(' ')+1:]

In [29]:
get_num_of_rooms('Двухкомнатная свердловский вариант')

'Двухкомнатная'

In [30]:
print(get_flat_type('Двухкомнатная свердловский вариант'))

свердловский вариант


In [31]:
data['num_of_rooms'] = data['type'].apply(get_num_of_rooms)
data['flat_type'] = data['type'].apply(get_flat_type)

In [32]:
data.head()

,id,type,district,adress,floor,total_square,living_square,kitchen_square,price,floor_num,total_floors,is_first_floor,is_last_floor,num_of_rooms,flat_type
0,1,Трехкомнатная улучшенная,Орджоникидзевский,Ленина пр-т 145/2,1/5,64.0,43.0,8.0,3750,1,5,1,0,Трехкомнатная,улучшенная
1,2,Трехкомнатная,Ленинский,Октябрьская 12,2/5,87.2,60.0,9.0,8300,2,5,0,0,Трехкомнатная,неизвестно
2,3,Однокомнатная нестандартная,Орджоникидзевский,Ленина пр-т 135,6/14,36.1,20.0,9.0,3330,6,14,0,0,Однокомнатная,нестандартная
3,4,Трехкомнатная нестандартная,Орджоникидзевский,Ленина пр-т 129,5/16,105.0,75.0,14.0,7700,5,16,0,0,Трехкомнатная,нестандартная
4,5,Двухкомнатная улучшенная,Орджоникидзевский,Сиреневый проезд 12,7/9,50.6,43.0,9.0,3800,7,9,0,0,Двухкомнатная,улучшенная


In [33]:
data['num_of_rooms'].isna().sum()

0

In [34]:
data['flat_type'].isna().sum()

0

In [35]:
data['num_of_rooms'].unique()

array(['Трехкомнатная', 'Однокомнатная', 'Двухкомнатная',
       'Четырехкомнатная', 'Многокомнатная'], dtype=object)

In [36]:
data['flat_type'].unique()

array(['улучшенная', 'неизвестно', 'нестандартная', 'брежневка',
       'хрущевка', 'хабаровский вариант', 'старой планировки',
       'свердловский вариант', 'распашонка', 'гребенка', 'раздельная',
       'сталинский вариант', 'малосемейка', 'смежная', 'евротрешка',
       'полусмежная', 'евродвушка', 'трапеция', 'ленинградский проект'],
      dtype=object)

### Улица

In [37]:
data[data['adress'] == ' '].shape

(4, 15)

In [38]:
data = data[data['adress'] != ' ']

In [39]:
# sorted(data['adress'].unique())
data['adress'].nunique()

376

In [40]:
def get_street(row):
    return row[: row.rfind(' ')]

In [41]:
get_street('ул Сталеваров 18/1',)

'ул Сталеваров'

In [42]:
data['street'] = data['adress'].apply(get_street)

In [43]:
data['street'].isna().sum()

0

In [44]:
data.head()

,id,type,district,adress,floor,total_square,living_square,kitchen_square,price,floor_num,total_floors,is_first_floor,is_last_floor,num_of_rooms,flat_type,street
0,1,Трехкомнатная улучшенная,Орджоникидзевский,Ленина пр-т 145/2,1/5,64.0,43.0,8.0,3750,1,5,1,0,Трехкомнатная,улучшенная,Ленина пр-т
1,2,Трехкомнатная,Ленинский,Октябрьская 12,2/5,87.2,60.0,9.0,8300,2,5,0,0,Трехкомнатная,неизвестно,Октябрьская
2,3,Однокомнатная нестандартная,Орджоникидзевский,Ленина пр-т 135,6/14,36.1,20.0,9.0,3330,6,14,0,0,Однокомнатная,нестандартная,Ленина пр-т
3,4,Трехкомнатная нестандартная,Орджоникидзевский,Ленина пр-т 129,5/16,105.0,75.0,14.0,7700,5,16,0,0,Трехкомнатная,нестандартная,Ленина пр-т
4,5,Двухкомнатная улучшенная,Орджоникидзевский,Сиреневый проезд 12,7/9,50.6,43.0,9.0,3800,7,9,0,0,Двухкомнатная,улучшенная,Сиреневый проезд


In [45]:
data['street'].nunique()

121

In [46]:
def get_street(row):
    dirt = ['ул. ', 'ул.', 'ул ', 'пр.']
    for dot in dirt:
        if dot in row:
            row = row.replace(dot, '')
    if row.find('. ') == 0:
        row = row[2:]
    return row

In [47]:
data['street'] = data['street'].apply(get_street)

In [48]:
streets_to_replace = {
    'зеленый лог' : 'Зеленый Лог',
    'Зеленый лог' : 'Зеленый Лог',
    'Зеленый лог 30 к' : 'Зеленый Лог',
    'Им. газеты \\"Правда\\"' : 'имени газеты Правда',
    'Им. газеты "Правда"' : 'имени газеты Правда',
    'проспект Сиреневый' : 'Сиреневый проезд',
    'карла маркса' : 'Карла Маркса',
    '50 лет Магнитки' : '50-летия Магнитки',
    'Ленина пр-т 210' : 'Ленина пр-т',
    'Советский переулок 12' : 'Советский переулок',
    '26 Горнолыжная' : 'Горнолыжная'
}

In [49]:
data['street'].replace(streets_to_replace, inplace=True)

In [50]:
data.shape

(438, 16)

In [51]:
data = data[(data['street'] != '133/1') & (data['street'] != '14')]

In [52]:
data.shape

(436, 16)

In [53]:
sorted(data['street'].unique())

['50-летия Магнитки',
 '70 лет Октября',
 'Анджиевского',
 'Болотникова',
 'Бориса Ручьева',
 'Вознесенская',
 'Вокзальная',
 'Ворошилова',
 'Габдрауфа Давлетова',
 'Гагарина',
 'Галиуллина',
 'Герцена',
 'Горнолыжная',
 'Горького',
 'Грязнова',
 'Доменщиков',
 'Жукова',
 'Завенягина',
 'Западное шоссе',
 'Зеленый Лог',
 'Индустриальная',
 'Казакова',
 'Калинина',
 'Калмыкова',
 'Карла Маркса',
 'Коробова',
 'Красноармейская',
 'Кронштадтская',
 'Куйбышева',
 'Курортная',
 'Ленина пр-т',
 'Ленинградская',
 'Лесная',
 'Лесопарковая',
 'Менделеева',
 'Металлургов',
 'Мичурина',
 'Молодежная',
 'Московская',
 'Набережная',
 'Николая Шишка',
 'Новая',
 'Октябрьская',
 'Оранжерейная',
 'Панькова',
 'Парковая',
 'Первомайская',
 'Пионерская',
 'Подольская',
 'Помяловского',
 'Привокзальная',
 'Пушкина',
 'Раздольная',
 'Ржевского переулок',
 'Российская',
 'Садовая',
 'Салтыкова-Щедрина',
 'Сиреневый проезд',
 'Советская',
 'Советский переулок',
 'Советской Армии',
 'Солнечный Берег',
 'Сочи

In [54]:
data['street'].nunique()

85

### Район

Посмотрим количество пропусков в столбце с данными о районе.

In [55]:
data['district'].isna().sum()

185

Заполним пропуски значением 'неизвестно'.

In [56]:
data['district'].fillna('неизвестно', inplace=True)

In [57]:
data['district'].isna().sum()

0

In [58]:
data['district'].unique()

array(['Орджоникидзевский', 'Ленинский', 'Правобережный', 'неизвестно',
       'Орджоникидзевский (левый берег)', 'ленинский',
       'Орджоникидзевский район', 'Орджо', 'правобережный'], dtype=object)

## Построение модели

### Подготовка обучающей и валидационной выборки.

Выделим характеристики, которые будем использовать для обучения модели.

|столбец|комментарий|
|:--|:--|
|id|не влияет на цену квартиры|
|type|вместо него будем использовать более информативные столбцы, сгенерированные на основе данных из этого столбца|
|**district**|может оказывать влияние|
|adress|вместо него будем использовать более информативные столбцы, сгенерированные на основе данных из этого столбца|
|floor|вместо него будем использовать более информативные столбцы, сгенерированные на основе данных из этого столбца|
|**total_square**|может оказывать влияние|
|**living_square**|может оказывать влияние|
|**kitchen_square**|может оказывать влияние|
|**price**|целевой признак|
|**floor_num**|может оказывать влияние|
|**total_floors**|может оказывать влияние|
|**is_first_floor**|может оказывать влияние|
|**is_last_floor**|может оказывать влияние|
|**num_of_rooms**|может оказывать влияние|
|**flat_type**|может оказывать влияние|
|**street**|может оказывать влияние|

In [59]:
features = data.drop(['id', 'type', 'adress', 'floor', 'price'], axis=1)

In [60]:
features.head()

,district,total_square,living_square,kitchen_square,floor_num,total_floors,is_first_floor,is_last_floor,num_of_rooms,flat_type,street
0,Орджоникидзевский,64.0,43.0,8.0,1,5,1,0,Трехкомнатная,улучшенная,Ленина пр-т
1,Ленинский,87.2,60.0,9.0,2,5,0,0,Трехкомнатная,неизвестно,Октябрьская
2,Орджоникидзевский,36.1,20.0,9.0,6,14,0,0,Однокомнатная,нестандартная,Ленина пр-т
3,Орджоникидзевский,105.0,75.0,14.0,5,16,0,0,Трехкомнатная,нестандартная,Ленина пр-т
4,Орджоникидзевский,50.6,43.0,9.0,7,9,0,0,Двухкомнатная,улучшенная,Сиреневый проезд


In [61]:
target = data['price']

In [62]:
target.head()

0    3750
1    8300
2    3330
3    7700
4    3800
Name: price, dtype: int64

In [63]:
features.isna().sum()

district          0
total_square      0
living_square     0
kitchen_square    0
floor_num         0
total_floors      0
is_first_floor    0
is_last_floor     0
num_of_rooms      0
flat_type         0
street            0
dtype: int64

Разобьем выборки на обучающую и валидационную в отношении 4 : 1.

In [64]:
features_train, features_val, target_train, target_val = train_test_split(
    features,
    target,
    test_size=0.2,
    random_state=RANDOM_STATE
)

In [65]:
features_train.isna().sum()

district          0
total_square      0
living_square     0
kitchen_square    0
floor_num         0
total_floors      0
is_first_floor    0
is_last_floor     0
num_of_rooms      0
flat_type         0
street            0
dtype: int64

### Предварительна обработка данных

Выделим категориальные и числовые признаки.

In [66]:
numeric = ['total_square', 'living_square', 'kitchen_square',
       'floor_num', 'total_floors', 'is_first_floor', 'is_last_floor']
categorical = ['district', 'num_of_rooms', 'flat_type', 'street']

Количество комнат попадает в категориальные признаки, потому что содержит значение «многоквартирная», которое нельзя заменить конкретным числовым значением.

Преобразуем категориальные признаки в числа с помощью порядкового кодирования. 

In [67]:
features_train['district'].isna().sum()

0

In [68]:
oe = OrdinalEncoder(
    handle_unknown='use_encoded_value',
    unknown_value = -1
)

In [69]:
cat_features_train = pd.DataFrame(oe.fit_transform(features_train[categorical]))
cat_features_train.columns = categorical

In [70]:
cat_features_train

,district,num_of_rooms,flat_type,street
0,2.0,3.0,6.0,28.0
1,2.0,2.0,6.0,22.0
2,0.0,0.0,14.0,38.0
3,7.0,3.0,6.0,68.0
4,7.0,2.0,6.0,58.0
...,...,...,...,...
343,2.0,0.0,6.0,46.0
344,2.0,0.0,6.0,17.0
345,0.0,0.0,6.0,73.0
346,7.0,2.0,6.0,28.0


In [71]:
features_train_oe =pd.merge(
    cat_features_train, 
    features_train[numeric].reset_index(drop=True), 
    left_index=True, 
    right_index=True)
features_train_oe

,district,num_of_rooms,flat_type,street,total_square,living_square,kitchen_square,floor_num,total_floors,is_first_floor,is_last_floor
0,2.0,3.0,6.0,28.0,65.0,0.0,9.0,1,5,1,0
1,2.0,2.0,6.0,22.0,30.0,17.0,6.0,10,10,0,1
2,0.0,0.0,14.0,38.0,43.0,29.0,6.0,3,5,0,0
3,7.0,3.0,6.0,68.0,60.1,40.0,15.0,2,2,0,1
4,7.0,2.0,6.0,58.0,31.0,18.0,6.0,5,9,0,0
...,...,...,...,...,...,...,...,...,...,...,...
343,2.0,0.0,6.0,46.0,69.0,40.0,15.0,1,1,1,1
344,2.0,0.0,6.0,17.0,78.0,62.7,15.0,2,3,0,0
345,0.0,0.0,6.0,73.0,62.8,38.0,9.0,4,5,0,0
346,7.0,2.0,6.0,28.0,35.7,25.0,10.0,1,1,1,1


In [72]:
target_train = target_train.reset_index(drop=True)

In [73]:
cat_features_val = pd.DataFrame(oe.transform(features_val[categorical]))
cat_features_val.columns = categorical

In [74]:
features_val_oe =pd.merge(
    cat_features_val, 
    features_val[numeric].reset_index(drop=True), 
    left_index=True, 
    right_index=True)
features_val_oe

,district,num_of_rooms,flat_type,street,total_square,living_square,kitchen_square,floor_num,total_floors,is_first_floor,is_last_floor
0,0.0,2.0,6.0,40.0,31.0,20.0,7.0,3,5,0,0
1,5.0,2.0,7.0,22.0,32.8,16.0,9.0,2,6,0,0
2,2.0,3.0,11.0,6.0,65.0,43.0,10.0,4,13,0,0
3,7.0,2.0,6.0,28.0,35.0,0.0,10.0,1,1,1,1
4,5.0,0.0,0.0,57.0,44.0,30.0,6.0,1,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...
83,3.0,3.0,13.0,51.0,68.0,53.0,6.0,2,5,0,0
84,0.0,2.0,13.0,40.0,33.0,20.0,7.0,3,5,0,0
85,7.0,2.0,6.0,36.0,42.6,20.0,15.0,2,3,0,0
86,7.0,3.0,6.0,71.0,69.0,42.0,9.0,4,9,0,0


In [75]:
target_val = target_val.reset_index(drop=True)

In [76]:
target_val

0     1950
1     3300
2     4450
3     1850
4     3000
      ... 
83    2750
84    2400
85    2900
86    5090
87    3300
Name: price, Length: 88, dtype: int64

Теперь масштабируем наши данные.

In [77]:
sscaler = StandardScaler()

In [80]:
features_train_sc = pd.DataFrame(sscaler.fit_transform(features_train_oe))
features_train_sc.columns = features_train_oe.columns

In [82]:
features_train_sc.head()

,district,num_of_rooms,flat_type,street,total_square,living_square,kitchen_square,floor_num,total_floors,is_first_floor,is_last_floor
0,-0.809395,1.018743,-0.295789,-0.439310,0.466990,-1.900083,0.003819,-1.074204,-0.340393,1.940908,-0.648389
1,-0.809395,0.280154,-0.295789,-0.715850,-1.091605,-0.921236,-0.719707,2.376469,1.104202,-0.515223,1.542284
2,-1.534412,-1.197023,2.030413,0.021588,-0.512699,-0.230284,-0.719707,-0.307387,-0.340393,-0.515223,-0.648389
3,1.003149,1.018743,-0.295789,1.404284,0.248787,0.403088,1.450871,-0.690796,-1.207150,-0.515223,1.542284
4,1.003149,0.280154,-0.295789,0.943385,-1.047074,-0.863656,-0.719707,0.459429,0.815283,-0.515223,-0.648389


In [83]:
features_val_sc = pd.DataFrame(sscaler.transform(features_val_oe))
features_val_sc.columns = features_val_oe.columns

In [84]:
features_val_sc.head()

,district,num_of_rooms,flat_type,street,total_square,living_square,kitchen_square,floor_num,total_floors,is_first_floor,is_last_floor
0,-1.534412,0.280154,-0.295789,0.113768,-1.047074,-0.748498,-0.478532,-0.307387,-0.340393,-0.515223,-0.648389
1,0.278132,0.280154,-0.005013,-0.715850,-0.966918,-0.978815,0.003819,-0.690796,-0.051474,-0.515223,-0.648389
2,-0.809395,1.018743,1.158088,-1.453287,0.466990,0.575826,0.244994,0.076021,1.970959,-0.515223,-0.648389
3,1.003149,0.280154,-0.295789,-0.439310,-0.868949,-1.900083,0.244994,-1.074204,-1.496069,1.940908,1.542284
4,0.278132,-1.197023,-2.040440,0.897295,-0.468167,-0.172705,-0.719707,-1.074204,-0.340393,1.940908,-0.648389


Модель линейной регрессии

In [86]:
class MyLinearRegression:
    
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T @ X) @ X.T @ y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [88]:
# наша модель
model_1 = MyLinearRegression()
model_1.fit(features_train_sc, target_train)
predictions = model_1.predict(features_val_sc)

# метрика MAE
mae = mean_absolute_error(target_val, predictions)
print(mae)

634.5584807828586


In [94]:
target_val

0     1950
1     3300
2     4450
3     1850
4     3000
      ... 
83    2750
84    2400
85    2900
86    5090
87    3300
Name: price, Length: 88, dtype: int64

In [96]:
for i in range(10):
    print(target_val.iloc[i], predictions.iloc[i])

1950 3079.5708495188946
3300 3057.038982494798
4450 5326.164712334555
1850 1783.7764015342673
3000 2604.4327598852187
4480 4327.519783622461
1 2488.4469108725207
1450 3914.5200374009346
4800 4305.119845470028
2700 3720.817867097765


In [102]:
from sklearn.tree import DecisionTreeRegressor

In [103]:
DTR = DecisionTreeRegressor(
    max_depth=10,
    random_state=RANDOM_STATE
)

In [104]:
DTR.fit(features_train_sc, target_train)

DecisionTreeRegressor(max_depth=10, random_state=12345)

In [105]:
predictionsDTR = DTR.predict(features_val_sc)

# метрика MAE
maeDTR = mean_absolute_error(target_val, predictionsDTR)
print(maeDTR)

626.1582185491276


In [106]:
from sklearn.ensemble import RandomForestRegressor

In [120]:
RFR = RandomForestRegressor(
    max_depth=10,
    random_state=RANDOM_STATE,
    n_estimators=100
)

In [121]:
RFR.fit(features_train_sc, target_train)

RandomForestRegressor(max_depth=10, random_state=12345)

In [122]:
predictionsRFR = RFR.predict(features_val_sc)

# метрика MAE
maeRFR = mean_absolute_error(target_val, predictionsRFR)
print(maeRFR)

507.82902876597376


In [124]:
for i in range(10):
    print(target_val.iloc[i], predictionsRFR[i])

1950 2292.8101526862833
3300 2498.5499134598663
4450 4869.418763084138
1850 1958.5750492562993
3000 3206.9168053492263
4480 4276.979048763542
1 2106.4340457256108
1450 4127.615744588744
4800 4182.890917599165
2700 2897.157001870083


In [ ]:
 3, 37, 82

In [89]:
example_1 = data[data['id'] == 3]

In [91]:
example_1['price']

2    3330
Name: price, dtype: int64

In [ ]:
model_1.predict(sscaler.transform())